<h1>Evaluations of NNet_Agent</h1> 

In [2]:
import os
import sys
import numpy as np

import sys

sys.path.append('..')

from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v0_fast_env
from pommerman.envs.v0 import Pomme
from pommerman.characters import Bomber
from pommerman import utility
from pommerman import my_utility

<h2>NNet_Agent</h2>

<h3>Overview:</h3>

<h4>NNet_Agent</h4>

* performs a MCTS at each action.
* selection (if all children available): using ucb $Qsa[(s, a)] + C * Ps[s][a] * math.sqrt(Ns[s]) / (1 + Nsa[(s, a)])$
* expansion (if not all children available): expand random unseen child. One expansion step expands 'my' and 'opponent' action.
* rollout: - <b>IF</b> node is terminal: get reward and backprobagate. <b>ELSE</b>: evaluate expanded node by nnet. Backprobagate Value from NNet.
* backprobagation: backprobagate rewards for 'me' and 'opponent' not the same. 'my' rewards update only 'my' Qsa, 'opponent' rewards update only 'opponent' Qsa.
* get action probability: return normalized distribution over $Nsa[(s, a)]$ (greedy: return maximum of $Nsa[(s, a)]$)

<h4>Learning process</h4>

* Self play
* Collect for each agent and each action: action-probs, canonical board state, reward (+rotated and mirrored)
* add to train examples
* start training the network
* evaluate network by playing old vs new network
* update if new nnet is better

In [7]:
# Passage = 0, Rigid = 1, Wood = 2, Bomb = 3, Flames = 4
# ExtraBomb = 6, IncrRange = 7, Kick = 8, Agent0 = 10, Agent1 = 11
game_type = 4

board = [[ 0,  0,  2,  1,  1,  1],
         [ 0,  0, 10,  0,  0,  0],
         [ 2,  0,  0,  1,  0,  1],
         [ 1,  0,  1,  0,  0,  1],
         [ 1,  0,  0,  0,  0,  1],
         [ 1, 11,  1,  1,  1,  0]]

game_state = my_utility.get_gamestate(board, [])
game_data = my_utility.get_gamedata(game_state, game_type)

my_utility.render_image(game_data)

KeyboardInterrupt: 

In [ ]:


<div>
<img src="attachment:grafik.png" width="200"/>
</div>

input

s = F.relu(self.bn1(self.conv1(s)))  # batch_size x num_channels x board_x x board_y

s = F.relu(self.bn2(self.conv2(s)))  # batch_size x num_channels x board_x x board_y

s = F.relu(self.bn3(self.conv3(s)))  # batch_size x num_channels x (board_x-2) x (board_y-2)

s = F.relu(self.bn4(self.conv4(s)))  # batch_size x num_channels x (board_x-4) x (board_y-4)

s = s.view(-1, self.num_channels * self.board_x * self.board_y)

s = F.dropout(F.relu(\text{self.fc_bn1}(self.fc1(s))), p=self.dropout, training=self.training)  # batch_size x 1024

s = F.dropout(F.relu(self.fc_bn2(self.fc2(s))), p=self.dropout, training=self.training)  # batch_size x 512

pi = self.fc3(s)  # batch_size x action_size

v = self.fc4(s)  # batch_size x 1

F.softmax(pi, dim=1), torch.tanh(v)

In [ ]:


<div>
<img src="attachment:grafik.png" width="200"/>
</div>

input

s = F.relu(self.bn1(self.conv1(s)))  # batch_size x num_channels x board_x x board_y

s = F.relu(self.bn2(self.conv2(s)))  # batch_size x num_channels x board_x x board_y

s = F.relu(self.bn3(self.conv3(s)))  # batch_size x num_channels x (board_x-2) x (board_y-2)

s = F.relu(self.bn4(self.conv4(s)))  # batch_size x num_channels x (board_x-4) x (board_y-4)

s = s.view(-1, self.num_channels * self.board_x * self.board_y)

s = F.dropout(F.relu(\text{self.fc_bn1}(self.fc1(s))), p=self.dropout, training=self.training)  # batch_size x 1024

s = F.dropout(F.relu(self.fc_bn2(self.fc2(s))), p=self.dropout, training=self.training)  # batch_size x 512

pi = self.fc3(s)  # batch_size x action_size

v = self.fc4(s)  # batch_size x 1

F.softmax(pi, dim=1), torch.tanh(v)